---

# CSCI 3202, Fall 2023
# Mancala Project - Outline

In [1]:
import copy, math, time, random

1. Play 100 games of random player against random player
    - What percentage of games does each player (1st or 2nd) win?
    - On average, how many moves does it take to win?
2. Build an AI player that uses minimax to choose the best move with a variable number of plies and a utility function we describe
    - What percentage of games does each player (AI or random) win?
    - On average, how many moves does it take to win?
3. Play 100 games with the random player against the minimax AI player at a depth of 5 plies
    - What percentage of games does each player (AI or random) win?
    - On average, how many moves does it take to win?
    - Is your AI player better than random chance? Write a paragraph or two describing why or why not.
4. Play 100 games with the random player against the Alpha-Beta AI player at a depth of 5 plies
    - How long does it take for a single game to run to completion?
    - What percentage of games does each player (AI or random) win?
    - On average, how many moves does it take to win?
    - Are your results for this part different from those for your minimax AI player? Write a paragraph or two describing why or why not.
5. (Extra Credit, 10 points). Play 100 games with the random player against the
    - Alpha-Beta AI player at a depth of 10 plies
    - How long does it take for a single game to run to completion?
    - What percentage of games does each player (AI or random) win?
    - On average, how many moves does it take to win?
    - Does increasing the number of plies improve the play for our AI player? Why or why not?

explanation for deepcopy - copy library
https://www.scaler.com/topics/copy-in-python/

In [2]:
class Mancala:
    def __init__(self, pits_per_player=6, stones_per_pit = 4):
        """
        The constructor for the Mancala class defines several instance variables:

        pits_per_player: This variable stores the number of pits each player has.
        stones_per_pit: It represents the number of stones each pit contains at the start of any game.
        board: This data structure is responsible for managing the Mancala board.
        current_player: This variable takes the value 1 or 2, as it's a two-player game, indicating which player's turn it is.
        moves: This is a list used to store the moves made by each player. It's structured in the format (current_player, chosen_pit).
        p1_pits_index: A list containing two elements representing the start and end indices of player 1's pits in the board data structure.
        p2_pits_index: Similar to p1_pits_index, it contains the start and end indices for player 2's pits on the board.
        p1_mancala_index and p2_mancala_index: These variables hold the indices of the Mancala pits on the board for players 1 and 2, respectively.
        """
        self.pits_per_player = pits_per_player
        self.board = [stones_per_pit] * ((pits_per_player+1) * 2)  # Initialize each pit with stones_per_pit number of stones 
        self.players = 2
        self.current_player = 1
        self.moves = []
        self.p1_pits_index = [0, self.pits_per_player-1]
        self.p1_mancala_index = self.pits_per_player
        self.p2_pits_index = [self.pits_per_player+1, len(self.board)-1-1]
        self.p2_mancala_index = len(self.board)-1
        
        # Zeroing the Mancala for both players
        self.board[self.p1_mancala_index] = 0
        self.board[self.p2_mancala_index] = 0

    def display_board(self):
        """
        Displays the board in a user-friendly format
        """
        player_1_pits = self.board[self.p1_pits_index[0]: self.p1_pits_index[1]+1]
        player_1_mancala = self.board[self.p1_mancala_index]
        player_2_pits = self.board[self.p2_pits_index[0]: self.p2_pits_index[1]+1]
        player_2_mancala = self.board[self.p2_mancala_index]

        print('P1               P2')
        print('     ____{}____     '.format(player_2_mancala))
        for i in range(self.pits_per_player):
            if i == self.pits_per_player - 1:
                print('{} -> |_{}_|_{}_| <- {}'.format(i+1, player_1_pits[i], 
                        player_2_pits[-(i+1)], self.pits_per_player - i))
            else:    
                print('{} -> | {} | {} | <- {}'.format(i+1, player_1_pits[i], 
                        player_2_pits[-(i+1)], self.pits_per_player - i))
            
        print('         {}         '.format(player_1_mancala))
        turn = 'P1' if self.current_player == 1 else 'P2'
        print('Turn: ' + turn)
        
    def valid_move(self, pit):
        """
        Function to check if the pit chosen by the current_player is a valid move.
        """
        
        if self.current_player == 1:
            if pit < self.p1_pits_index[0] or pit > self.p1_pits_index[1]:
                return False
            if self.board[pit] == 0:
                return False
        # write your code here
        elif self.current_player == 2:
            if pit < self.p2_pits_index[0] or  pit > self.p2_pits_index[1]:
                return False
            if self.board[pit] == 0:
                return False
            
        return True
        
    def random_move_generator(self):
        """
        Function to generate random valid moves with non-empty pits for the random player
        """
        
        valid_pits = []
        if self.current_player == 1:
            valid_pits = [pit + 1 for pit in range(self.p1_pits_index[0], self.p1_pits_index[1]+1) if self.board[pit] > 0]
        if self.current_player == 2:
            valid_pits = [pit - self.p2_pits_index[0] + 1 for pit in range(self.p2_pits_index[0], self.p2_pits_index[1]+1) if self.board[pit] > 0]
        
        if valid_pits:
            pit = random.choice(valid_pits)
        else:
            return -1
        
        return pit
    
    def play(self, pit):
        """
        This function simulates a single move made by a specific player using their selected pit. It primarily performs three tasks:
        1. It checks if the chosen pit is a valid move for the current player. If not, it prints "INVALID MOVE" and takes no action.
        2. It verifies if the game board has already reached a winning state. If so, it prints "GAME OVER" and takes no further action.
        3. After passing the above two checks, it proceeds to distribute the stones according to the specified Mancala rules.

        Finally, the function then switches the current player, allowing the other player to take their turn.
        """
        
        # write your code here

        if self.current_player == 1:
            index = pit - 1
        else:
            index = self.p2_pits_index[0] + pit - 1

        if not self.valid_move(index):
            print("INVALID MOVE")
            return self.board
        if self.winning_eval() != None:
            print("GAME OVER")
            return self.board

        distribute_stones = self.board[index]
        self.board[index] = 0

        while distribute_stones > 0:
            index = (index + 1) % len(self.board)
            
            if self.current_player == 1 and index == self.p2_mancala_index:
                index = (index + 1) % len(self.board)
            if self.current_player == 2 and index == self.p1_mancala_index:
                index = (index + 1) % len(self.board)
                
            self.board[index] += 1
            distribute_stones -= 1
        
        if self.current_player == 1 and self.p1_pits_index[0] <= index <= self.p1_pits_index[1] and self.board[index] == 1:
            opp_index = self.p2_pits_index[1] - index
            if self.board[opp_index] > 0:
                self.board[self.p1_mancala_index] += self.board[index] + self.board[opp_index]
                self.board[index] = 0
                self.board[opp_index] = 0
        
        elif self.current_player == 2 and self.p2_pits_index[0] <= index <= self.p2_pits_index[1] and self.board[index] == 1:
            opp_index = self.p1_pits_index[1] - (index - self.p2_pits_index[0])
            if self.board[opp_index] > 0:
                self.board[self.p2_mancala_index] += self.board[index] + self.board[opp_index]
                self.board[index] = 0
                self.board[opp_index] = 0
        
        self.moves.append((self.current_player, pit))
        
        if self.current_player == 1:
            self.current_player = 2
        else:
            self.current_player = 1
        
        return self.board
    
    def winning_eval(self):
        """
        Function to verify if the game board has reached the winning state.
        Hint: If either of the players' pits are all empty, then it is considered a winning state.
        """
        
        p1_pits = self.board[self.p1_pits_index[0]: self.p1_pits_index[1]+1]
        p2_pits = self.board[self.p2_pits_index[0]: self.p2_pits_index[1]+1]
        p1_mancala = self.board[self.p1_mancala_index]
        p2_mancala = self.board[self.p2_mancala_index]
        
        # write your code here
        if all(stones == 0 for stones in p1_pits):
            if p1_mancala > p2_mancala:
                return 1
            elif p1_mancala < p2_mancala:
                return 2
            else:
                return 0
        elif all(stones == 0 for stones in p2_pits):
            if p1_mancala > p2_mancala:
                return 1
            elif p1_mancala < p2_mancala:
                return 2
            else:
                return 0
        else:
            return None

In [3]:
class MancalaAI:
    def __init__(self, depth, state):
        self.depth = depth
        self.state = state
    
    def minimax(self, state, depth, maximizing_player, cur_player):
        if depth == 0 or state.winning_eval() is not None:
            return self.evaluate_state(state)
        
        if state.current_player == 1:
            possible_valid_moves = [pit + 1 for pit in range(state.p1_pits_index[0], state.p1_pits_index[1]+1) if state.valid_move(pit)]
        else:
            possible_valid_moves = [pit - state.p2_pits_index[0] + 1 for pit in range(state.p2_pits_index[0], state.p2_pits_index[1]+1) if state.valid_move(pit)]
        
        if maximizing_player:
            # generate all possible states for the maximizing player, and recurse 
            # until you reach the stop condition - terminal state
            max_eval = float('-inf')
            for move in possible_valid_moves:
                next_state = Mancala()
                next_state.board = state.board.copy()
                next_state.current_player = state.current_player
                next_state.play(move)
                value = self.minimax(next_state, depth-1, False, 3 - cur_player) # minimizing players' move
                max_eval = max(max_eval, value)
            return max_eval
        else:
            # generate all possible states for the maximizing player, and recurse
            min_eval = float('inf')
            for move in possible_valid_moves:
                next_state = Mancala()
                next_state.board = state.board.copy()
                next_state.current_player = state.current_player
                next_state.play(move)
                value = self.minimax(next_state, depth-1, True, 3 - cur_player) # maximizing players' move
                min_eval = min(min_eval, value)
            return min_eval
        
    
    def minimax_alpha_beta(self, state, depth, alpha, beta, maximizing_player, cur_player):
        if depth == 0 or state.winning_eval() is not None:
            return self.evaluate_state(state)
        
        if state.current_player == 1:
            possible_valid_moves = [pit + 1 for pit in range(state.p1_pits_index[0], state.p1_pits_index[1]+1) if state.valid_move(pit)]
        else:
            possible_valid_moves = [pit - state.p2_pits_index[0] + 1 for pit in range(state.p2_pits_index[0], state.p2_pits_index[1]+1) if state.valid_move(pit)]
        
        if maximizing_player:
            max_eval = float('-inf')
            for move in possible_valid_moves:
                next_state = Mancala()
                next_state.board = state.board.copy()
                next_state.current_player = state.current_player
                next_state.play(move)
                value = self.minimax_alpha_beta(next_state, depth-1, alpha, beta, False, 3 - cur_player)
                max_eval = max(max_eval, value)
                if max_eval > beta:
                    break
                alpha = max(alpha, max_eval)
            return max_eval
        else:
            min_eval = float('inf')
            for move in possible_valid_moves:
                next_state = Mancala()
                next_state.board = state.board.copy()
                next_state.current_player = state.current_player
                next_state.play(move)
                value = self.minimax_alpha_beta(next_state, depth-1, alpha, beta, True, 3 - cur_player)
                min_eval = min(min_eval, value)
                if min_eval <= alpha:
                    break
                beta = min(beta, min_eval)
            return min_eval

    def best_move(self, state, alpha_beta):
        if state.current_player == 1:
            possible_valid_moves = [pit + 1 for pit in range(state.p1_pits_index[0], state.p1_pits_index[1]+1) if state.valid_move(pit)]
        else:
            possible_valid_moves = [pit - state.p2_pits_index[0] + 1 for pit in range(state.p2_pits_index[0], state.p2_pits_index[1]+1) if state.valid_move(pit)]

        best_move = None
        best_eval = float('-inf')

        for move in possible_valid_moves:
            next_state = Mancala()
            next_state.board = state.board.copy()
            next_state.current_player = state.current_player
            next_state.play(move)

            if state.current_player == 1:
                if alpha_beta == True:
                    value = self.minimax_alpha_beta(next_state, self.depth, float('-inf'), float('inf'), True, state.current_player)
                else:
                    value = self.minimax(next_state, self.depth, True, state.current_player)
                    
                # find the max value out of all options, and return the move corresponding to that max value
                if value > best_eval:
                    best_eval = value
                    best_move = move
            else:
                if alpha_beta == True:
                    value = self.minimax_alpha_beta(next_state, self.depth, float('-inf'), float('inf'), False, state.current_player)
                else:
                    value = self.minimax(next_state, self.depth, False, state.current_player)
               
            
                # find the min value out of all options, and return the move corresponding to that min value
                if value < best_eval:
                    best_eval = value
                    best_move = move

        return best_move

    def evaluate_state(self, state):
        # Utility function  :- Difference between P1 mancala and p2 mancala
        return state.board[state.p1_mancala_index] - state.board[state.p2_mancala_index]

### 1. Random vs. Random 100 Games

In [10]:
#Play 100 games of random player against random player
p1_count = 0
p2_count = 0
total_moves = 0

for i in range(1,101):
    game = Mancala()
    
    while(game.winning_eval() == None):
        #Random Player 1
        if game.current_player == 1:
            p1_pit = game.random_move_generator()
            game.play(p1_pit)
        else:
            p2_pit = game.random_move_generator()
            game.play(p2_pit)
    
    winner = game.winning_eval()
    total_moves += len(game.moves)
    
    if winner == 1:
        p1_count += 1
    if winner == 2:
        p2_count += 1
    

#Determine win percentage for each player, and average number of moves for each game
p1_avg = p1_count / 100
p2_avg = p2_count / 100
move_avg = total_moves/100

print("Average amount of moves: " + str(move_avg))
print("Player 1 win percent: " + str(p1_avg * 100) + " Player 2 win percent: " + str(p2_avg * 100))

Average amount of moves: 43.1
Player 1 win percent: 51.0 Player 2 win percent: 47.0


### 2. Minimax vs. Random 100 Games

In [6]:
ai_count = 0
random_count = 0
total_moves = 0
depth = 5

for i in range(1,101):
    game = Mancala()
    ai = MancalaAI(depth, game)
    
    while(game.winning_eval() == None):
        #AI Player 
        if game.current_player == 1:
            move = ai.best_move(game, False)
            game.play(move)
        else:
            move = game.random_move_generator()
            game.play(move)
    
    winner = game.winning_eval()
    total_moves += len(game.moves)
    
    if winner == 1:
        ai_count += 1
    if winner == 2:
        random_count += 1
    
ai_avg = ai_count / 100
random_avg = random_count / 100
move_avg = total_moves/100

print("Average amount of moves: " + str(move_avg))
print("AI win percent: " + str(ai_avg * 100) + " Random win percent: " + str(random_avg * 100))

Average amount of moves: 31.86
AI win percent: 97.0 Random win percent: 2.0


### 3. Alpha-Beta vs. Random 100 Games

In [7]:
ai_count = 0
random_count = 0
total_moves = 0
depth = 5

for i in range(1,101):
    game = Mancala()
    ai = MancalaAI(depth, game)
    
    start_time = time.time()
    
    while(game.winning_eval() == None):
        #AI Player 
        if game.current_player == 1:
            move = ai.best_move(game, True)
        else:
            move = game.random_move_generator()
    
        game.play(move)
    
    end_time = time.time()
    
    winner = game.winning_eval()
    total_moves += len(game.moves)
    
    if winner == 1:
        ai_count += 1
    if winner == 2:
        random_count += 1
    
ai_avg = ai_count / 100
random_avg = random_count / 100
move_avg = total_moves/100

print("Average amount of moves: " + str(move_avg))
print("AI win percent: " + str(ai_avg * 100) + " Random win percent: " + str(random_avg * 100))
print("Time taken to complete 1 game: " + str(end_time - start_time))

Average amount of moves: 32.01
AI win percent: 99.0 Random win percent: 1.0
Time taken to complete 1 game: 0.07578134536743164


### 4. Extra Credit

In [11]:
ai_count = 0
random_count = 0
total_moves = 0
depth = 10

for i in range(1,101):
    game = Mancala()
    ai = MancalaAI(depth, game)
    
    start_time = time.time()
    
    while(game.winning_eval() == None):
        #AI Player 
        if game.current_player == 1:
            move = ai.best_move(game, True)
        else:
            move = game.random_move_generator()
    
        game.play(move)
    
    end_time = time.time()
    
    winner = game.winning_eval()
    total_moves += len(game.moves)
    
    if winner == 1:
        ai_count += 1
    if winner == 2:
        random_count += 1
    
ai_avg = ai_count / 100
random_avg = random_count / 100
move_avg = total_moves/100

print("Average amount of moves: " + str(move_avg))
print("AI win percent: " + str(ai_avg * 100) + " Random win percent: " + str(random_avg * 100))
print("Time taken to complete 1 game: " + str(end_time - start_time))

Average amount of moves: 34.19
AI win percent: 98.0 Random win percent: 2.0
Time taken to complete 1 game: 8.463171482086182
